In [64]:
import pandas as pd
import numpy as np
import time
import streamlit as st
import functools

In [14]:
board_size = 19
win_length = 5
player = 1
score = {1: 2, -1: 1}

In [3]:
tab  = [[0 for _ in range(board_size)] for _ in range(board_size)]

In [70]:
%cd ..
from src.heuristic import heuristic
%load_ext autoreload
%autoreload 2

/
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
valeurs_possibles = [-1, 0, 1]

# Générer 100 tableaux 19x19
tableaux = [np.random.choice(valeurs_possibles, size=(19, 19)) for _ in range(100)]

# Affichage d'un exemple de tableau
print(tableaux[0])

[[-1 -1  0 -1  0  0 -1  0  1  1  0  0  1 -1 -1 -1  0  1  1]
 [-1  1  1  0 -1  1 -1  0  0 -1  0 -1 -1  0  0 -1  1  1  1]
 [-1  0  1 -1  0  1 -1  1  0 -1  1  1  0  0  1 -1  1  1  1]
 [-1  0 -1 -1 -1 -1  0 -1  0 -1  0  1  0 -1 -1  0 -1  0 -1]
 [ 1  1  0 -1 -1  0  1  1  1 -1  1  1  0  0  0  0  1  1  0]
 [-1 -1  1  1 -1  0  1  0  0  1  1  1  1  1 -1  0  1  1 -1]
 [ 0  0  0  1  0  0  1  1 -1  1  0  1  0 -1  1 -1  0  1 -1]
 [ 1 -1  1 -1  0  1  0  0  0  0  0  0  0  0  0  1  1 -1 -1]
 [-1 -1 -1 -1  1  0  1  0  0  1 -1  1  0 -1  1  1  1  1  0]
 [ 1 -1  1 -1  1 -1  0  1 -1  0  1 -1 -1  0 -1  1  1  1  0]
 [ 0  0 -1  0  0 -1  0  0  1  1  0 -1 -1 -1 -1  0  1  1  1]
 [-1 -1  1  1  0  1 -1  0  1  0 -1  0  0  1 -1 -1 -1  0  0]
 [ 1  0  1 -1  1  0 -1 -1  1  1  1  1 -1  1  1 -1  1  1 -1]
 [ 1  0  0 -1  0  1  1  0  0  1  0  0 -1 -1  0  1 -1  0 -1]
 [ 0 -1 -1  1  1  0 -1 -1  1  0 -1 -1  0  0  0  0  0  1  0]
 [-1  1  1 -1  0  1 -1  1 -1  0  1  0  1  0  1  1  1  0  1]
 [ 1  0  1  1 -1  0  0  1  0  0  0  1 -1

In [85]:
directions = [(-1, -1),
            (-1, 0),
            (-1, 1),
            (0, -1),
            (0, 1),
            (1, -1),
            (1, 0),
            (1, 1)
            ]

In [86]:
def heuristic_row(board_size, win_len, tab, player, reward):
    i = 0
    result = 0
    while i < board_size:
        j = 0
        while j <= board_size - 1:
            if tab[j][i] == player:
                open = 0
                if j - 1 >= 0:
                    if tab[j - 1][i] == 0:
                        open = 1
                k = 0
                while k < win_len and j < board_size and tab[j][i] == player:
                    k += 1
                    j += 1
                if j < board_size:
                    if tab[j][i] == 0:
                        open += 1
                result += reward[open][k]
            elif tab[j][i] == -player:
                open = 0
                if j - 1 >= 0:
                    if tab[j - 1][i] == 0:
                        open = 1
                k = 0
                while k < win_len and j < board_size and tab[j][i] == -player:
                    k += 1
                    j += 1
                if j < board_size:
                    if tab[j][i] == 0:
                        open += 1
                result -= reward[open][k] * 1.5
            else:
                j += 1
        i += 1
    return result

def heuristic_diag(board_size, win_len, tab, player, reward):
    result = 0
    j =  1 - board_size
    while j < board_size:
        i = max(0, j)
        while i < min(board_size, j + board_size):
            if tab[i][i - j] == player:
                    open = 0
                    if i - 1 >= 0 and i - 1 - j >= 0:
                        if tab[i - 1][i - 1 - j] == 0:
                            open = 1
                    k = 0
                    while k < win_len and i < min(board_size, j + board_size) and tab[i][i - j] == player:
                        k += 1
                        i += 1
                    if i < board_size and i - j < board_size:
                        if tab[i][i - j] == 0:
                            open += 1
                    result += reward[open][k]
            elif tab[i][i - j] == -player:
                    open = 0
                    if i - 1 >= 0 and i - 1 - j >= 0:
                        if tab[i - 1][i - 1 - j] == 0:
                            open = 1
                    k = 0
                    while k < win_len and i < min(board_size, j + board_size) and tab[i][i - j] == -player:
                        k += 1
                        i += 1
                    if i < board_size and i - j < board_size:
                        if tab[i][i - j] == 0:
                            open += 1
                    result -= reward[open][k] * 1.5
            else:
                i += 1
        j += 1
    return result

def check_alignement_capture(board, row, col, player):
    for dr, dc in directions:
        r, c = row + dr, col + dc
        if 0 <= r < len(board) and 0 <= c < len(board):
            if board[r][c] == -player:  # Found an opponent piece
                # search in the same direction for another opponent piece
                r2, c2 = r + dr, c + dc
                if 0 <= r2 < len(board) and 0 <= c2 < len(board):
                    if board[r2][c2] == -player:
                        # check if the next cell in the same direction is player's piece
                        r3, c3 = r2 + dr, c2 + dc
                        if 0 <= r3 < len(board) and 0 <= c3 < len(board):
                            if board[r3][c3] == player:
                                return True
    return False

def heuristic_capture(tab, player, row, col, g_score, reward_capture):
    res = 0
    if check_alignement_capture(tab, row, col, player):
        res += reward_capture[g_score[player]]
    if check_alignement_capture(tab, row, col, -player):
        res -= reward_capture[g_score[-player]] + 1
    return res

def get_reward(win_len):
    reward_closed = [0] * (win_len + 1)
    reward_open1 = [0] + [0] + [10**i for i in range(-2, win_len, 2)]
    reward_open2 = [0] + [0] + [10**i for i in range(-1, win_len + 1, 2)]
    reward_closed[win_len] = 10**(win_len)
    reward_open1[win_len] = 10**(win_len)
    reward = [reward_closed, reward_open1, reward_open2]
    reward_capture = [10 + (i+1) for i in range(5)]
    reward_capture[4] = 10**(win_len)
    return reward, reward_capture

def heuristic_test(board_size, win_len, tab, player, row, col, g_score):
    reward, reward_capture = get_reward(win_len)
    tab = np.array(tab)
    res = heuristic_capture(tab, player, row, col, g_score, reward_capture)
    res += heuristic_row(board_size, win_len, tab, player, reward)
    res += heuristic_row(board_size, win_len, tab, player, reward)
    res += heuristic_diag(board_size, win_len, tab, player, reward)
    res += heuristic_diag(board_size, win_len, tab, player, reward)
    return res

In [1]:
tableaux = [np.random.choice(valeurs_possibles, size=(19, 19)) for _ in range(1000)]
start = time.time()
print("Start")
for i in range(1000):
    res = heuristic_test(board_size, win_length, tableaux[i], player, 2, 2, score)
end = time.time()
elapsed = end -start
print(f"End = {elapsed:.2}ms")

NameError: name 'np' is not defined

init : End = 0.78ms

cache :